In [146]:
import requests
import json
import numpy as np
import datetime
import string
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
import statsmodels.graphics.tsaplots as sgt

from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import mean_squared_error

import itertools

from statsmodels.tsa.statespace.sarimax import SARIMAX
from prophet import Prophet

# paquetes para importar
import requests
import json
import numpy as np
import datetime
import string
import pandas as pd

import xgboost as xgb
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score, KFold
import plotly.express as px
import plotly.figure_factory as ff
import plotly.graph_objs as go
import chart_studio
import chart_studio.plotly as py 

In [147]:
df_resume = pd.read_csv('demanda_electrica_diaria.csv')

In [148]:
df_resume['date'] = pd.DatetimeIndex(df_resume['date']).date
df_resume['year'] = pd.DatetimeIndex(df_resume['date']).year
df_resume['month'] = pd.DatetimeIndex(df_resume['date']).month
df_resume.head()

,date,value,weekday,working_day,year,month
0,2014-01-01,3315035.0,2,0,2014,1
1,2014-01-02,4059590.0,3,1,2014,1
2,2014-01-03,4098986.0,4,1,2014,1
3,2014-01-04,3867911.0,5,1,2014,1
4,2014-01-05,3648038.0,6,1,2014,1


In [149]:
X, y =  df_resume.loc[:,['month', 'year']].values, df_resume.loc[:,'value'].values
data_dmatrix = xgb.DMatrix(X,label=y)

In [150]:
from sklearn.model_selection import train_test_split

# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.12, random_state=123)

In [151]:
reg_mod = xgb.XGBRegressor(objective='reg:squarederror',
    n_estimators=1000,
    learning_rate=0.10,
    subsample=0.5,
    colsample_bytree=1, 
    max_depth=5,
)
reg_mod.fit(X_train, y_train)
XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.1, max_delta_step=0, max_depth=5,
             min_child_weight=1, monotone_constraints='()',
             n_estimators=1000, n_jobs=8, num_parallel_tree=1, random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=0.5,
             tree_method='exact', validate_parameters=1, verbosity=None)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
             gamma=0, gpu_id=-1, importance_type='gain',
             interaction_constraints='', learning_rate=0.1, max_delta_step=0,
             max_depth=5, min_child_weight=1, missing=nan,
             monotone_constraints='()', n_estimators=1000, n_jobs=8,
             num_parallel_tree=1, predictor=None, random_state=0, reg_alpha=0,
             reg_lambda=1, scale_pos_weight=1, subsample=0.5,
             tree_method='exact', validate_parameters=1, verbosity=None)

In [152]:
reg_mod.fit(X_train,y_train)

predictions = reg_mod.predict(X_test)

In [153]:
rmse = np.sqrt(mean_squared_error(y_test, predictions))
print("RMSE: %f" % (rmse))

from sklearn.metrics import r2_score
r2 = np.sqrt(r2_score(y_test, predictions))
print("R_Squared Score : %f" % (r2))

from sklearn.metrics import mean_absolute_percentage_error
mape = mean_absolute_percentage_error(y_test, predictions)
print("MAPE : %f" % (mape))

RMSE: 334806.174613
R_Squared Score : 0.545231
MAPE : 0.071181


In [154]:
len(predictions)

351

In [155]:
len(y_test)

351

In [158]:
df_resume.tail(351)

,date,value,weekday,working_day,year,month
2571,2021-01-15,4975157.0,4,1,2021,1
2572,2021-01-16,4429765.0,5,1,2021,1
2573,2021-01-17,4114808.0,6,1,2021,1
2574,2021-01-18,4805672.0,0,1,2021,1
2575,2021-01-19,4903743.0,1,1,2021,1
...,...,...,...,...,...,...
2917,2021-12-27,3686647.0,0,1,2021,12
2918,2021-12-28,3729169.0,1,1,2021,12
2919,2021-12-29,3692921.0,2,1,2021,12
2920,2021-12-30,3689439.0,3,1,2021,12


In [159]:
# Creating an empty Dataframe with column names only
result = pd.DataFrame(columns=['Time', 'Test', 'Predicted'])
result['Time'] = pd.date_range(start='2021-01-15', end='2021-12-31', freq='D')
result['Test'] = y_test
result['Predicted'] = predictions
#Using Plotly to build the graph
fig = go.Figure()
fig.add_trace(go.Scatter(x=result['Time'], y=result['Test'],
                    mode='lines',
                    name='Test'))
fig.add_trace(go.Scatter(x=result['Time'], y=result['Predicted'],
                    mode='lines',
                    name='Predicted'))

# Edit the layout
fig.update_layout(title='Test vs Predicted Values',
                   xaxis_title='Months',
                   yaxis_title='Electricity (GWh)',
                   template='gridon')

fig.show()